In [1]:
import duckdb, os

os.chdir('../../../../')

In [2]:
path_database = 'database/dmeyf.db'
path_file_raw = 'datasets/raw/competencia_02_crudo.csv'
path_file_processed = 'datasets/processed/competencia_02.csv'
path_file_foto_reporte = 'datasets/processed/competencia_02_foto_reporte.csv'
path_file_clase_ternaria = 'datasets/processed/competencia_02.csv'

In [3]:
con = duckdb.connect(path_database)

con.sql(f"""
        CREATE OR REPLACE TABLE competencia_02 AS (
            SELECT
                *
            FROM read_csv_auto('{path_file_raw}')
        );
        """
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [14]:
con.sql(
    """
    SELECT *
    FROM competencia_02
    --WHERE numero_de_cliente = 29183981
    LIMIT 10;
    """
)

┌───────────────────┬──────────┬────────────────┬───┬────────────────┬──────────────────────┬──────────────────┐
│ numero_de_cliente │ foto_mes │ active_quarter │ … │ Visa_cconsumos │ Visa_cadelantosefe…  │ Visa_mpagominimo │
│       int64       │  int64   │     int64      │   │     int64      │        int64         │      double      │
├───────────────────┼──────────┼────────────────┼───┼────────────────┼──────────────────────┼──────────────────┤
│          29183981 │   201901 │              1 │ … │             54 │                    0 │          5008.71 │
│          29184630 │   201901 │              1 │ … │             10 │                    0 │          1395.87 │
│          29185587 │   201901 │              1 │ … │              2 │                    0 │              0.0 │
│          29185646 │   201901 │              1 │ … │             31 │                    0 │          2897.31 │
│          29187499 │   201901 │              1 │ … │              3 │                    0 │   

In [8]:
con.sql(
    """
    DESCRIBE SELECT * FROM competencia_02;
    """
)

┌─────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│       column_name       │ column_type │  null   │   key   │ default │  extra  │
│         varchar         │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ numero_de_cliente       │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ foto_mes                │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ active_quarter          │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ cliente_vip             │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ internet                │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ cliente_edad            │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ cliente_antiguedad      │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ mrentabilidad           │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ mrentabilidad_

In [29]:
# Calculate lag features using foto_mes as date column and numero_de_cliente as group column
con.sql(
    """
    SELECT
        *,
        LAG(Visa_cconsumos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS Visa_cconsumos_lag_1,
        LAG(Visa_cconsumos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS Visa_cconsumos_lag_2,
        LAG(Visa_cconsumos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS Visa_cconsumos_lag_3,
        LAG(Visa_cconsumos, 4) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS Visa_cconsumos_lag_4,
        
        LAG(mrentabilidad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS mrentabilidad_lag_1,
        LAG(mrentabilidad, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS mrentabilidad_lag_2,
        LAG(mrentabilidad, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS mrentabilidad_lag_3,
        LAG(mrentabilidad, 4) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS mrentabilidad_lag_4
    FROM competencia_02
    ORDER BY numero_de_cliente, foto_mes;
    """
)
            
    

┌───────────────────┬──────────┬────────────────┬──────────────────────┬──────────────────────┐
│ numero_de_cliente │ foto_mes │ Visa_cconsumos │ Visa_cconsumos_lag_1 │ Visa_cconsumos_lag_2 │
│       int64       │  int64   │     int64      │        int64         │        int64         │
├───────────────────┼──────────┼────────────────┼──────────────────────┼──────────────────────┤
│          30069432 │   201901 │             17 │                 NULL │                 NULL │
│          30069432 │   201902 │             18 │                   17 │                 NULL │
│          30069432 │   201903 │             26 │                   18 │                   17 │
│          30069432 │   201904 │             42 │                   26 │                   18 │
│          30069432 │   201905 │             19 │                   42 │                   26 │
│          30069432 │   201906 │             28 │                   19 │                   42 │
│          30069432 │   201907 │        

In [ ]:
|